# Training a convolutional network for anatomy guided PET image denoising and deblurring

In this tutorial, we will learn how to set up and train a simple 3D convolutional network for anatomical-guided denoising and deblurring of PET images.

We will set up a network that takes a batch of 3D tensors with 2 channels (e.g. PET and MR) as input and outputs a batch of 3D tensors with 1 channel (denoised and deblurred PET image).


The approach and the model architecture that we will use in this tutorial is inspired by Schramm et al., ["Approximating anatomically-guided PET reconstruction in image space using a convolutional neural network"](https://doi.org/10.1016/j.neuroimage.2020.117399), NeuroImage 2021, DOI 10.1016/j.neuroimage.2020.117399

![foo bar](https://raw.githubusercontent.com/gschramm/pyapetnet/master/figures/fig_1_apetnet.png)



This tutotial uses simulated PET/MR data based on the brain web. However, applying the same training strategy to real data should be straight forward. 

To setup and train the model, we will use tensorflow and keras to show the basic concepts of setting up and training a model. Of course, the same concepts can be used with any other deep learning frame work (such as e.g. pytorch).

**The tutorial is split into two notebooks:**
1. In the first notebook ([01_tf_data.ipynb](01_tf_data.ipynb)), we will learn how to setup an data loader pipeline to efficiently create mini batches of training data including data augmentation.
2. In the second notebook ([02_tf_models.ipynb](02_tf_models.ipynb)), we will learn how to setup and train the model architecture shown above.

Finally, it wil be you turn to combine the knowledge of these two notebooks to train your own network.

To run the tutorial, we recommend to install the python package ```pyapetnet v.1.1``` (or later). This will install all depdencies that we need (e.g. ```tensorflow``` and ```nibabel```). In order to visualize the created model in the 2nd notebook, you also need to install ```pydot``` and ```graphviz```. All packages are available on pypi.